In [225]:
from collections import Counter
import matplotlib.pyplot as plt
import glob
import hashlib
import io
import math
import os
import re
import csv
import datetime
from TwitterAPI import TwitterAPI
%matplotlib inline

In [208]:
def get_data():
    path = 'data'
    filenames = []
    for filename in os.listdir(path):
        if ".csv" in filename:
            filenames.append(path + os.sep +filename)
    filenames = sorted(filenames)
    
    return sorted(filenames)

filenames = get_data()
print '%d traffic files found' % len(filenames)


160 traffic files found


In [219]:
def get_file_data(filenames):
    traffic_data = []
    for filename in filenames:
        f = open(filename, 'rb')
        reader = csv.reader(f)
        row_count = -1
        for row in reader:
            if row_count != -1:
                traffic_data.append(row)
            row_count += 1
        f.close()
    
    return traffic_data

traffic_data = get_file_data(filenames)
print len(traffic_data)

4640


In [291]:
traffic_data[0]

['Rogers Park - West Ridge',
 '1',
 '-87.709645',
 '-87.654561',
 '41.997946',
 '42.026444',
 'North of Devon. Kedzie to Lake Shore',
 '20.45',
 '2015-10-31 16:50:40.0']

In [224]:
CURRENT_SPEED = 7
LAST_UPDATED = 8
traffic_data[0][LAST_UPDATED]

'2015-10-31 16:50:40.0'

In [228]:
#convert 'LAST_UPDATED' param of traffic_data to a datetime object and return in format YYYY,MM,DD,H,M,S
def get_date_obj(date_string):
    date_string = date_string[:-2]
    date_obj = datetime.datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S")    
    return date_obj
get_date_obj(traffic_data[0][LAST_UPDATED])

datetime.datetime(2015, 10, 31, 16, 50, 40)

In [256]:
#divide the tweets according to time. morning: 0700 - 1100, noon 1100 - 1400, evening 1400 - 1700, night: 1700-2100
def divide_trafficdata_bytime(traffic_data,LAST_UPDATED):
    morning = []
    noon = []
    evening = []
    night = []
    late_night = []
    traffic_bytime = {}
    for row in traffic_data:
        date = get_date_obj(row[LAST_UPDATED])
        if date.time().hour >= 7 and date.time().hour < 11:
            morning.append(row)
        elif date.time().hour >= 11 and date.time().hour < 15:
            noon.append(row)
        elif date.time().hour >= 15 and date.time().hour < 17:
            evening.append(row)
        elif date.time().hour >= 17 and date.time().hour < 21:
            night.append(row)
        else:
            late_night.append(row)
            
    traffic_bytime['morning'] = morning
    traffic_bytime['noon'] = noon
    traffic_bytime['evening'] = evening
    traffic_bytime['night'] = night
    traffic_bytime['late_night'] = late_night
    
    return traffic_bytime

In [289]:
import numpy as np
def get_average_speed(traffic_data,CURRENT_SPEED):
    
    speed = [row[CURRENT_SPEED] for row in traffic_data]
    speed = np.array(speed,dtype='float')
    print np.mean(speed)
    print np.min(speed)
    print np.max(speed)
get_average_speed(traffic_bytime['morning'],CURRENT_SPEED)
get_average_speed(traffic_bytime['noon'],CURRENT_SPEED)
get_average_speed(traffic_bytime['evening'],CURRENT_SPEED)
get_average_speed(traffic_bytime['night'],CURRENT_SPEED)

23.1714389102
0.0
173.86
23.2482327586
0.0
41.86
20.8666190076
10.23
43.64
21.3287715517
0.0
37.94


**TWITTER**

In [294]:
import tweepy

# Replace the API_KEY and API_SECRET with your application's key and secret.
auth = tweepy.AppAuthHandler('aTpLPRCEcym54l0kgzTHH8VTt', 'jbXrNKePxA11hgY0y1fHHfphNH9wBKbymnmgUaMHhiprIEhFzG')
 
api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
 
if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)

SSLError: EOF occurred in violation of protocol (_ssl.c:590)

In [43]:
import sys
import jsonpickle
import os

searchQuery = 'chicago traffic'  # this is what we're searching for
maxTweets = 10000000 # Some arbitrary large number
tweetsPerQry = 100  # this is the max the API permits
fName = 'tweets.txt' # We'll store the tweets in a text file.


# If results from a specific ID onwards are reqd, set since_id to that ID.
# else default to no lower limit, go as far back as API allows
sinceId = None

# If results only below a specific ID are, set max_id to that ID.
# else default to no upper limit, start from the most recent tweet matching the search query.
max_id = -1L

tweetCount = 0
print("Downloading max {0} tweets".format(maxTweets))
with open(fName, 'w') as f:
    while tweetCount < maxTweets:
        try:
            if (max_id <= 0):
                if (not sinceId):
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry)
                else:
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            since_id=sinceId)
            else:
                if (not sinceId):
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            max_id=str(max_id - 1))
                else:
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            max_id=str(max_id - 1),
                                            since_id=sinceId)
            if not new_tweets:
                print("No more tweets found")
                break
            for tweet in new_tweets:
                f.write(jsonpickle.encode(tweet._json, unpicklable=False) +
                        '\n')
            tweetCount += len(new_tweets)
            print("Downloaded {0} tweets".format(tweetCount))
            max_id = new_tweets[-1].id
        except tweepy.TweepError as e:
            # Just exit if any error
            print("some error : " + str(e))
            break

print ("Downloaded {0} tweets, Saved to {1}".format(tweetCount, fName))

Downloaded 100 tweets
Downloaded 200 tweets
Downloaded 300 tweets
Downloaded 400 tweets
Downloaded 500 tweets
Downloaded 600 tweets
Downloaded 700 tweets
Downloaded 800 tweets
Downloaded 900 tweets
Downloaded 1000 tweets
Downloaded 1100 tweets
Downloaded 1200 tweets
Downloaded 1300 tweets
Downloaded 1400 tweets
Downloaded 1500 tweets
Downloaded 1600 tweets
Downloaded 1700 tweets
Downloaded 1800 tweets
Downloaded 1900 tweets
Downloaded 2000 tweets
Downloaded 2100 tweets
Downloaded 2200 tweets
Downloaded 2300 tweets
Downloaded 2400 tweets
Downloaded 2500 tweets
Downloaded 2600 tweets
Downloaded 2700 tweets
Downloaded 2800 tweets
Downloaded 2900 tweets
Downloaded 3000 tweets
Downloaded 3100 tweets
Downloaded 3200 tweets
Downloaded 3300 tweets
Downloaded 3400 tweets
Downloaded 3428 tweets
No more tweets found
Downloaded 3428 tweets, Saved to tweets.txt


In [292]:


searchQuery = 'chicago traffic'  # this is what we're searching for
maxTweets = 10000000 # Some arbitrary large number
tweetsPerQry = 100  # this is the max the API permits
geocode = '42.026444,-87.654561,2mi'
fName = 'tweets_geo.txt' # We'll store the tweets in a text file.


# If results from a specific ID onwards are reqd, set since_id to that ID.
# else default to no lower limit, go as far back as API allows
sinceId = None

# If results only below a specific ID are, set max_id to that ID.
# else default to no upper limit, start from the most recent tweet matching the search query.
max_id = -1L

tweetCount = 0
print("Downloading max {0} tweets".format(maxTweets))
with open(fName, 'w') as f:
    while tweetCount < maxTweets:
        try:
            if (max_id <= 0):
                if (not sinceId):
                    new_tweets = api.search(q=searchQuery, geocode = geocode,count=tweetsPerQry)
                else:
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            since_id=sinceId)
            else:
                if (not sinceId):
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            max_id=str(max_id - 1))
                else:
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            max_id=str(max_id - 1),
                                            since_id=sinceId)
            if not new_tweets:
                print("No more tweets found")
                break
            for tweet in new_tweets:
                f.write(jsonpickle.encode(tweet._json, unpicklable=False) +
                        '\n')
            tweetCount += len(new_tweets)
            print("Downloaded {0} tweets".format(tweetCount))
            max_id = new_tweets[-1].id
        except tweepy.TweepError as e:
            # Just exit if any error
            print("some error : " + str(e))
            break

print ("Downloaded {0} tweets, Saved to {1}".format(tweetCount, fName))

NameError: name 'tweepy' is not defined

In [290]:
import jsonpickle
f = open('tweets.txt')
all_tweets = []
for line in f:
    t = jsonpickle.decode(line)
    all_tweets.append(t)
    
print all_tweets[0]

{u'contributors': None, u'truncated': False, u'text': u'The Latest: Chicago schools prepare lesson plan on shooting syndication.ap.orgProtesters block rush hour traffic a\u2026 https://t.co/zCOPRBv4BN', u'is_quote_status': False, u'in_reply_to_status_id': None, u'in_reply_to_user_id': None, u'id': 669942638393585664L, u'favorite_count': 0, u'entities': {u'symbols': [], u'user_mentions': [], u'hashtags': [], u'urls': [{u'indices': [116, 139], u'url': u'https://t.co/zCOPRBv4BN', u'expanded_url': u'http://ift.tt/1kVrO7G', u'display_url': u'ift.tt/1kVrO7G'}]}, u'retweeted': False, u'coordinates': None, u'source': u'<a href="http://ifttt.com" rel="nofollow">IFTTT</a>', u'in_reply_to_screen_name': None, u'id_str': u'669942638393585664', u'retweet_count': 0, u'metadata': {u'iso_language_code': u'en', u'result_type': u'recent'}, u'favorited': False, u'user': {u'follow_request_sent': None, u'has_extended_profile': False, u'profile_use_background_image': True, u'id': 3270783145L, u'verified': Fa

In [16]:
print all_tweets[0]['text']

The Latest: Chicago schools prepare lesson plan on shooting syndication.ap.orgProtesters block rush hour traffic a… https://t.co/zCOPRBv4BN


In [17]:
all_tweets[0]['user']['name']
all_tweets[0]['user']['location']
from collections import Counter
counts = Counter()
for tweet in all_tweets:
    counts.update(tweet['text'].lower().split())
print('%d unique words in %d tweets' % (len(counts), len(all_tweets)) )
counts.most_common(10)


6645 unique words in 3428 tweets


[(u'traffic', 2596),
 (u'chicago', 2140),
 (u'in', 1841),
 (u'rt', 1411),
 (u'the', 1260),
 (u'at', 920),
 (u'to', 800),
 (u'on', 644),
 (u'#chicago', 598),
 (u'is', 579)]

In [110]:
import re

def tokenize(string, lowercase, keep_punctuation,
             collapse_urls, collapse_mentions):
    """ Split a tweet into tokens."""
    if not string:
        return []
    if lowercase:
        string = string.lower()
    tokens = []
    if collapse_urls:
        string = re.sub('http\S+', '', string)
    if collapse_mentions:
        string = re.sub('@\S+', '', string)
    if keep_punctuation:
        tokens = string.split()
    else:
        tokens = re.sub('\W+', ' ', string).split()
    return tokens

In [111]:
tokens_list = tokenize(all_tweets[0]['text'], lowercase=True,
                            keep_punctuation=False,
                            collapse_urls=False, collapse_mentions=False)
print tokens_list
print(len(tokens_list))

[u'the', u'latest', u'chicago', u'schools', u'prepare', u'lesson', u'plan', u'on', u'shooting', u'syndication', u'ap', u'orgprotesters', u'block', u'rush', u'hour', u'traffic', u'a', u'https', u't', u'co', u'zcoprbv4bn']
21


In [129]:
score = []
for tweet in all_tweets:
    pscore = 0
    nscore = 0
    tokens_list = tokenize(tweet['text'], lowercase=True,
                            keep_punctuation=False,
                            collapse_urls=False, collapse_mentions=False)
    for token in tokens_list:
        word = swn.senti_synsets(token)
        if word:
            pscore += word[0].pos_score()
            nscore += word[0].neg_score()
            score.append((pscore,nscore))
    
print score[0]

(0.0, 0.0)


In [178]:
#convert 'created_at' param of tweet to a datetime object and return in format YYYY,MM,DD,H,M,S
def get_date_object(tweet):
    ti= tweet['created_at'].split()
    newt = ti[0]+" "+ti[1]+" "+ti[2]+" "+ti[3]+" "+ti[5]
    d = datetime.datetime.strptime(newt, "%a %b %d %H:%M:%S %Y")    
    return d
get_date_object(all_tweets[0])

datetime.datetime(2015, 11, 26, 18, 15, 46)

In [201]:
#divide the tweets according to time. morning: 0700 - 1100, noon 1100 - 1400, evening 1400 - 1700, night: 1700-2100

def divide_tweets_time(all_tweets):
    morning = []
    noon = []
    evening = []
    night = []
    late_night = []
    tweets_bytime = {}
    for tweet in all_tweets:
        tweet_date = get_date_object(tweet)
        if tweet_date.time().hour >= 7 and tweet_date.time().hour < 11:
            morning.append(tweet)
        elif tweet_date.time().hour >= 11 and tweet_date.time().hour < 14:
            noon.append(tweet)
        elif tweet_date.time().hour >= 14 and tweet_date.time().hour < 17:
            evening.append(tweet)
        elif tweet_date.time().hour >= 17 and tweet_date.time().hour < 21:
            night.append(tweet)
        else:
            late_night.append(tweet)
            
    tweets_bytime['morning'] = morning
    tweets_bytime['noon'] = noon
    tweets_bytime['evening'] = evening
    tweets_bytime['night'] = night
    tweets_bytime['late_night'] = late_night
    
    return tweets_bytime
    

In [202]:
tweets_bytime = divide_tweets_time(all_tweets)
print len(tweets_bytime['morning'])
print len(tweets_bytime['noon'])
print len(tweets_bytime['evening'])
print len(tweets_bytime['night'])
print len(tweets_bytime['late_night'])

latenight = tweets_bytime['late_night']
print latenight[-1]['created_at']

135
280
399
602
2012
Tue Nov 17 21:14:29 +0000 2015


In [57]:
from nltk.corpus import sentiwordnet as swn
# Depends on nltk (pip install nltk)
# See http://www.nltk.org/data.html
happy = swn.senti_synsets('good traffic','a')
print happy

[]


In [75]:
for tweet in all_tweets:
    print '======================'
    text = tweet2tokens(t, lowercase=True,
                            keep_punctuation=False,
                            collapse_urls=True, collapse_mentions=True)
    pscore = 0
    nscore = 0
    for t in text:
        word = swn.senti_synsets(t)
        if word:
            pscore += word[0].pos_score()
            nscore += word[0].neg_score()
    print pscore,nscore,text,tweet['created_at']

0.125 2.125 [u'chicago', u'cops', u'so', u'above', u'the', u'law', u'they', u'r', u'willing', u'2', u'abuse', u'even', u'those', u'who', u'investigate', u'them', u'why', u'can', u't', u'bad', u'cops', u'be', u'fired'] Thu Nov 26 18:15:46 +0000 2015


TypeError: string indices must be integers

In [133]:
from senti_classifier import senti_classifier
t = all_tweets[0:100]
for tweet in t:
    sentence = tweet['text']
    pos_score, neg_score = senti_classifier.polarity_scores(sentence)
    #if pos_score != 0.0 or neg_score!=0.0:
    print pos_score,neg_score

0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0


KeyboardInterrupt: 